In [ ]:
%%capture
!pip install fyers-apiv2

### **Fyers Authentication**
**All the required parameters for creating auth_code are stored in the fyers_secret.txt file in the given order.That is client id in first line followed by secret_key in next line and so on**

In [ ]:
# Using readlines()
file1 = open('/content/drive/MyDrive/fyers_secret.txt', 'r')
Lines = file1.readlines()
client_id = Lines[0][0:-1]
secret_key = Lines[1][0:-1]
redirect_uri = Lines[2][0:-1]
response_type = Lines[3][0:-1]
grant_type = Lines[4][0:-1]
state = Lines[5]
# Strips the newline character
state

'success'

In [ ]:
from fyers_api import fyersModel
from fyers_api import accessToken
#import asyncio
from concurrent import futures
session=accessToken.SessionModel(client_id=client_id,
secret_key=secret_key,redirect_uri=redirect_uri, 
response_type=response_type, grant_type=grant_type,
state=state)

response = session.generate_authcode()  
#response

moduleName: fyersService, ERR: cannot import name 'future_set_exception_unless_cancelled' from 'tornado.concurrent' (/usr/local/lib/python3.7/dist-packages/tornado/concurrent.py)


In [ ]:
auth_code = ""
session.set_token(auth_code)
response = session.generate_token()
access_token = response["access_token"]
log_path = "/content/drive/MyDrive/Fyers_logs"
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token,log_path=log_path)
is_async = True  #(By default the async will be False, Change to True for async API calls.)

In [ ]:
data = {"symbols":"NSE:NIFTY2261616000CE"}
o = fyers.quotes(data)#['d'][0]#.get('v')#.get('ask')
o

{'d': [{'n': 'NSE:NIFTY2261616000CE',
   's': 'ok',
   'v': {'ask': 304.2,
    'bid': 303.8,
    'ch': -191.9,
    'chp': -38.66,
    'cmd': {'c': 304.5,
     'h': 305.15,
     'l': 303.95,
     'o': 305.15,
     't': 1654845480,
     'tf': '12:48',
     'v': 2300},
    'description': 'NSE:NIFTY2261616000CE',
    'exchange': 'NSE',
    'fyToken': '101122061643703',
    'high_price': 369.9,
    'low_price': 296.05,
    'lp': 304.5,
    'open_price': 353.35,
    'original_name': 'NSE:NIFTY2261616000CE',
    'prev_close_price': 496.4,
    'short_name': 'NIFTY2261616000CE',
    'spread': 0.4,
    'symbol': 'NSE:NIFTY2261616000CE',
    'tt': 1654819200,
    'volume': 1695950}}],
 's': 'ok'}

## **Intraday Cash Future Arbitrage Detector**

#### **This intraday cash future arbitrage detector takes advantage of the minute fluctuations in the basis of the asset in a real time live environment and generates high frequency buy/sell signals accordingly. These minute fluctuations are detected in a relatively stable market using bollinger bands. Every day until maturity will have some mean basis, but when the markets are stable around mid day there will be minute fluctuations in this basis where it will suddenly rise/fall and then go back to the mean basis level in the next millisecond, we make use of these minute fluctuations and make riskless profits.**
#### **REQUIREMENTS: This strategy needs APIs which are capable of placing orders at high frequencies. The markets need to be relatively stable, this strategy fails in highly fluctuating markets.**
#### **There are 2 cases possible:**
#### **(i) Contango (Basis equals future - spot): If the basis crosses the upper band then we short the basis(= short future + long spot) and vice versa in the opposite scenario. We close our positions in the next second when the basis reaches its mean level.**
#### **(ii) Backwardation (Basis equals spot - future): If the basis crosses the upper band then we short the basis(= short spot + long future) and vice versa in the opposite scenario. We close our positions in the next second when the basis reaches its mean level.**

In [ ]:
import numpy as np
import pandas as pd
#import yfinance as yf
import time
import math
#import mplfinance as mpf
from IPython.display import clear_output 
import warnings
import datetime
import pytz 
from datetime import date
import json
from IPython.display import clear_output

In [ ]:
def get_symbol(index_fut):
  allowedSegment = [11]
  futurl = 'https://public.fyers.in/sym_details/NSE_FO.csv'
  futurl_symbollist = pd.read_csv(futurl , header = None)
  futurl_symbollist.columns =  ['FyersToken', 'Name', 'Instrument', 'lot', 'tick', 'ISIN','TradingSession', 'Lastupdatedate', 'Expirydate', 'Symbol', 'Exchange', 'Segment','ScripCode','ScripName','No idea','STRIKE','OPT_TYPE']
  futurl_symbollist =futurl_symbollist[futurl_symbollist['Instrument'].isin(allowedSegment)]
  ind_list = [i for i in range(len(futurl_symbollist))]
  futurl_symbollist = futurl_symbollist.reset_index( inplace =  False )
  fut_symbol = []
  for i in range(len(futurl_symbollist)):
    if(str(futurl_symbollist['Symbol'].iloc[i][0:9]) == index_fut[0:9]):
      fut_symbol.append(str(futurl_symbollist['Symbol'].iloc[i]))
  return fut_symbol
symbols = get_symbol("NSE:NIFTY50-INDEX")
symbols

['NSE:NIFTY22JUNFUT', 'NSE:NIFTY22JULFUT', 'NSE:NIFTY22AUGFUT']

In [ ]:

def get_bb(data, lookback, n):  ##### For calculating bollinger bands for the basis

  middle_bb = []
  upper_bb = []
  lower_bb = []
  std = []
  for i in range(lookback,len(data)):
      middle_bb.append(sum(data[i-lookback:i])/lookback)
      variance = sum([((x - middle_bb[i-lookback]) ** 2) for x in data[i-lookback:i]]) / lookback
      res = variance ** 0.5
      std.append(res)
      
  
  for j in range(len(std)):
      upper_bb.append(middle_bb[j] + std[j] * n)
      lower_bb.append(middle_bb[j] - std[j] * n)

  return upper_bb, middle_bb, lower_bb

In [ ]:
from fyers_api.Websocket import ws
from os.path import exists
import math

index_fut = 'NSE:NIFTY50-INDEX'
symbol = get_symbol(index_fut)[0]
symbols = []
symbols.append(symbol)
symbols.append(index_fut)
#print(symbols)
access_tok = client_id + ':' + access_token
data_type = 'symbolData'
run_background  = False #(by default) If you don’t want to get the orderUpdate over the console you can set the run_background to True
str1 = 0
str2 = 0
time_str_fut = ""
time_str_spot = ""
basis1 = 0
order_long = False
order_short = False
basis_list = []
#(This is the custom message function which we need to have in order to receive the Symbol/order update and accordingly you can manipulate the data you receive through this function  )
def custom_message(msg):
      global str1,str2,net_p1,net_p2,basis_list,time_str_fut,time_str_spot
      msg = msg[0]
      
      ##### Storing the future and index object in different variables - str1 and str2
      if(str(msg['symbol'][-3:]) == 'FUT'):
        str1 = msg
      
      if(str(msg['symbol'][0:11]) == 'NSE:NIFTY50'):
        str2 = msg 
      
      if((str1!=0) and (str2!=0)): # To check for existence of the objects
      
        if(abs(int(str1['timestamp'])-int(str2['timestamp']))<=1): # To work on objects which have time stamp of equal to or less than one
          fut_price = float(str1['ltp'])
          spot_price = float(str2['ltp'])
          fut_true_price = fut_price*math.exp(-0.0485*22/365)
          transaction_cost = 0.0
         
          if(float(fut_true_price-spot_price-transaction_cost)>0):#############     CONTANGO    #############
            basis = (fut_price-spot_price-transaction_cost)
            if(len(basis_list)<30):
                basis_list.append(basis)
            else:
              for i in range(len(basis_list)-1):
                basis_list[i] = basis_list[i+1]
              basis_list[len(basis_list)-1] = basis

              upper_bb,middle_bb,lower_bb = get_bb(basis_list, 20, 2)
          
            if(len(upper_bb)>=10):
                
                """
                if(order_short==True): print("Short basis: Time(for futures):    "+str(time_str_fut)+ "   Time(for spot): "+str(time_str_spot)+"  Profit:   " + str(basis1-basis))
                #if(order_long==True): print("Long basis:  Time(for futures):   "+str(time_str_fut)+ "   Time(for spot): "+str(time_str_spot)+"  Profit:   " + str(basis-basis1))
                """
                time_str_fut = str1['timestamp']
                time_str_spot = str2['timestamp']
                
                print("basis at: " + str(basis_list[29]) +"  Time(for futures):    "+str(time_str_fut)+"   Time(for spot): "+str(time_str_spot) + " spot cash at avg price  " + str(spot_price) +" and fut at  " + str(fut_price) )
              
                if((float(upper_bb[9])<float(basis_list[29])) and (float(upper_bb[8])>float(basis_list[28]))):  # To check for those values which crosses the upper band from below
                    
                    print("Short basis at: " + str(basis_list[29]) +"  Time(for futures):    "+str(time_str_fut)+"   Time(for spot): "+str(time_str_spot)+ " short fut at avg price  " + str(fut_price) +"  and long spot cash   " + str(spot_price) )
                    #order_short = True
                    #basis1 = basis_list[29]
                    
                if((float(lower_bb[9])>float(basis_list[29])) and (float(lower_bb[8])<float(basis_list[28]))):  # To check for those values which crossed the lower band from above
                    
                    print("Long basis at: " + str(basis_list[29])  +"  Time(for futures):    "+str(time_str_fut)+"   Time(for spot): "+str(time_str_spot)+ " long Fut at avg price  " + str(fut_price) +"  and short spot cash   " + str(spot_price) )
                    #order_long = True
                    #basis1 = basis_list[29]

          if((spot_price-fut_true_price-transaction_cost)>0):#############     BACKWARDATION    #############
            basis = (spot_price-fut_true_price-transaction_cost)
            if(len(basis_list)<30):
                basis_list.append(basis)
            else:
              for i in range(len(basis_list)-1):
                basis_list[i] = basis_list[i+1]
              basis_list[len(basis_list)-1] = basis
            
              upper_bb,middle_bb,lower_bb = get_bb(basis_list, 20, 2)
            
            if(len(upper_bb)>=10):
            
                """
                if(order_short==True): print("Short basis: Time(for futures):    "+str(time_str_fut)+ "   Time(for spot): "+str(time_str_spot)+"  Profit:   " + str(basis1-basis))
                #if(order_long==True): print("Long basis:  Time(for futures):   "+str(time_str_fut)+ "   Time(for spot): "+str(time_str_spot)+"  Profit:   " + str(basis-basis1))
                """
                time_str_fut = str1['timestamp']
                time_str_spot = str2['timestamp']
                
                print("basis at: " + str(basis_list[29]) +"  Time(for futures):    "+str(time_str_fut)+"   Time(for spot): "+str(time_str_spot) + " spot cash at avg price  " + str(spot_price) +" and fut at  " + str(fut_price) )
             
                if((float(upper_bb[9])<float(basis_list[29])) and (float(upper_bb[8])>float(basis_list[28]))):  # To check for those values which crosses the upper band from below
                    
                    print("Short basis at: " + str(basis_list[29]) +" Time(for futures):    "+str(time_str_fut)+"   Time(for spot): "+str(time_str_spot)+ " short spot at avg price  " + str(spot_price) +"  and long fut at   " + str(spot_price) )
                    #order_short = True
                    #basis1 = basis_list[29]
                    
                if((float(lower_bb[9])>float(basis_list[29])) and (float(lower_bb[8])<float(basis_list[28]))):  # To check for those values which crossed the lower band from above
                    
                    print("Long basis at: " + str(basis_list[29])  +" Time(for futures):    "+str(time_str_fut)+"   Time(for spot): "+str(time_str_spot)+ " long spot at avg price  " + str(spot_price) +"  and short spot cash   " + str(fut_price) )
                    #order_long = True
                    #basis1 = basis_list[29]          

        
fyersSocket = ws.FyersSocket(access_token=access_tok,run_background=run_background,log_path="/content/drive/MyDrive/Fyers_logs")
fyersSocket.websocket_data = custom_message
fyersSocket.subscribe(symbol=symbols,data_type=data_type)
fyersSocket.keep_running()
#fyersSocket.unsubscribe(symbol = symbol)

basis at: 37.125678797399814  Time(for futures):    1654845936   Time(for spot): 1654845935 spot cash at avg price  16219.75 and fut at  16230.0
Long basis at: 37.125678797399814 Time(for futures):    1654845936   Time(for spot): 1654845935 long spot at avg price  16219.75  and short spot cash   16230.0
basis at: 38.67713830691173  Time(for futures):    1654845937   Time(for spot): 1654845938 spot cash at avg price  16221.8 and fut at  16230.5
basis at: 38.2284518654742  Time(for futures):    1654845938   Time(for spot): 1654845938 spot cash at avg price  16221.8 and fut at  16230.95
basis at: 38.178451865474926  Time(for futures):    1654845938   Time(for spot): 1654845939 spot cash at avg price  16221.75 and fut at  16230.95
basis at: 38.12859781642692  Time(for futures):    1654845939   Time(for spot): 1654845939 spot cash at avg price  16221.75 and fut at  16231.0
basis at: 37.92859781642619  Time(for futures):    1654845939   Time(for spot): 1654845940 spot cash at avg price  1622

KeyboardInterrupt: ignored